# Runtime Test

In [1]:
import os
import sys
import json
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
from termcolor import colored
import cv2
import time
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# change this property
NOMEROFF_NET_DIR = "/var/www/nomeroff-net/"
MASK_RCNN_DIR = "/var/www/Mask_RCNN/"

MASK_RCNN_LOG_DIR = os.path.join(NOMEROFF_NET_DIR, "logs/")
MASK_RCNN_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/mask_rcnn_numberplate_0700.h5")
REGION_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/imagenet_vgg16_np_region_2019_1_18.h5")
FILL_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/imagenet_vgg16_np_filled_2019_1_15.h5")

sys.path.append(NOMEROFF_NET_DIR)

from NomeroffNet import  filters, RectDetector, TextDetector, RegionDetector, Detector, textPostprocessing, FillDetector

nnet = Detector(MASK_RCNN_DIR, MASK_RCNN_LOG_DIR)
nnet.loadModel(MASK_RCNN_MODEL_PATH)

rectDetector = RectDetector()
textDetector = TextDetector()
regionDetector = RegionDetector()
regionDetector.load(REGION_MODEL_PATH)
fillDetector = FillDetector()
fillDetector.load(FILL_MODEL_PATH)

Using TensorFlow backend.


Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [2]:
def test(dirName, fname):
    img_path = os.path.join(dirName, fname)
    img = mpimg.imread(img_path)
    img_w = img.shape[1]
    img_h = img.shape[0]
    if img_w > 1920:
        img = cv2.resize(img, (1920, int(1920/img_w*img_h)))
    NP = nnet.detect([img])
    cv_img_masks = filters.cv_img_mask(NP) 
    nGood = 0
    nBad = 0
    res = []
    for img_mask in cv_img_masks:
        points = rectDetector.detect(img_mask, fixRectangleAngle=1, outboundWidthOffset=3)
        zone = rectDetector.get_cv_zones(img, points)
        regionId = regionDetector.predict(zone)
        regionName = regionDetector.getLabels(regionId)
        text = textDetector.detect(zone)
        text = textPostprocessing(text, regionName)
        res.append(text)

In [3]:
N = 10

i = 0
j = 0
start_time = time.time()
rootDir = 'images/'
for i in np.arange(N):
    for dirName, subdirList, fileList in os.walk(rootDir):
        for fname in fileList:
            test(dirName, fname)
            j += 1
    i += 1
    print(i/N)
end_time = time.time() - start_time

0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0


### Intel(R) Core(TM) i5-7500 CPU @ 3.40GHz

In [4]:
print(f"Processed {j} photos")
print(f"Time {end_time}")
print(f"One photo process {end_time/j} seconds")

Processed 260 photos
Time 1931.2203931808472
One photo process 7.427770743003259 seconds
